In [1]:
#!nvidia-smi

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torchsummary import summary

from DatasetLogReturn import StockDatasetLogReturn
from tqdm.notebook import tqdm

from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
import random as rd

In [3]:
configs = {
    "batch_size" : 128,
    "max_file_count" : 2179,
    "train_file" : 25,
    "validation_file" : 7,
    "test_file" : 15,
    "input_size" : 5,
    "output_size" : 1,
    "init_learning_rate" : 1e-2,
    "scheduler_factor": 0.2,
    "scheduler_patience": 2,
    "scheduler_min_lr": 1e-4,
    "epoch" : 4
}

configs["file_count"] = configs["train_file"] + configs["validation_file"] + configs["test_file"]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
tickers = list(map(str.strip,open("tickers.txt","r").readlines()))
filenames = ["./data/" + x + ".csv" for x in tickers]
rd.shuffle(filenames)
filenames = filenames[:configs["file_count"]]

train_files, test_files = train_test_split(filenames,test_size=configs["test_file"]/configs["file_count"])
train_files, val_files = train_test_split(train_files,test_size=configs["validation_file"]/(configs["train_file"] + configs["validation_file"]))

train_dataset = StockDatasetLogReturn(train_files, configs["input_size"], configs["output_size"])
train_loader = DataLoader(train_dataset,batch_size=configs["batch_size"],shuffle=True)

val_dataset = StockDatasetLogReturn(val_files,configs["input_size"],configs["output_size"])
val_loader = DataLoader(val_dataset,batch_size=configs["batch_size"],shuffle=True)

test_dataset = StockDatasetLogReturn(test_files,configs["input_size"],configs["output_size"])
test_loader = DataLoader(test_dataset,batch_size=configs["batch_size"],shuffle=True)

In [5]:
torch.set_default_dtype(torch.float32)
class StockLSTM(nn.Module) :
    def __init__(self) :
        super().__init__()
        self.hidden_size = 256
        self.batch_size = configs["batch_size"]
        self.h0 = torch.zeros((4,self.batch_size,self.hidden_size)).to(device)
        self.c0 = torch.zeros((4,self.batch_size,self.hidden_size)).to(device)
        self.layer1 = nn.LSTM(1,self.hidden_size,4,dropout=0.2,batch_first=True)
        self.layer2 = nn.Sequential(
            nn.Linear(self.hidden_size,64),
            nn.ReLU()
        )
        self.layer3 = nn.Linear(64,1)

    def forward(self, input) :
        h1,(hn,cn) = self.layer1(input,(self.h0,self.c0))
        h2 = self.layer2(h1[:,-1,:])
        output = self.layer3(h2)
        return output


In [6]:
model = StockLSTM()
model = model.to(device)

summary(model,input_size=(configs["batch_size"],configs["input_size"],1))

Layer (type:depth-idx)                   Param #
├─LSTM: 1-1                              1,844,224
├─Sequential: 1-2                        --
|    └─Linear: 2-1                       16,448
|    └─ReLU: 2-2                         --
├─Linear: 1-3                            65
Total params: 1,860,737
Trainable params: 1,860,737
Non-trainable params: 0


Layer (type:depth-idx)                   Param #
├─LSTM: 1-1                              1,844,224
├─Sequential: 1-2                        --
|    └─Linear: 2-1                       16,448
|    └─ReLU: 2-2                         --
├─Linear: 1-3                            65
Total params: 1,860,737
Trainable params: 1,860,737
Non-trainable params: 0

In [7]:
optimizer = torch.optim.Adam(model.parameters(), lr = configs["init_learning_rate"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    'min',
    factor=configs['scheduler_factor'],
    patience=configs['scheduler_patience'],
    min_lr=configs['scheduler_min_lr']
)
loss_fn = torch.nn.L1Loss()

In [8]:
train_losses = []
val_losses = []
model_folder = "/models/LSTM/"
best_loss = 1000000

for epoch in tqdm(range(configs["epoch"])) :

    model.train()

    cur_train_loss = []
    for input,target in tqdm(train_loader) :

        close_price_input = input[:,4,:,None]
        close_price_target = target[:,4,:]
        close_price_input = close_price_input.to(device)
        close_price_target = close_price_target.to(device)

        if(close_price_input.shape[0] != configs["batch_size"]) :
            continue

        optimizer.zero_grad()

        output = model(close_price_input)

        loss = loss_fn(output,close_price_target)

        loss.backward()

        optimizer.step()

        train_losses.append(loss)
        cur_train_loss.append(loss)

    avarage_train_loss = np.mean(np.array(cur_train_loss))

    model.eval()

    with torch.no_grad() :
        cur_val_loss = []
        for input,target in tqdm(val_loader) :

            close_price_input = input[:,4,:,None]
            close_price_target = target[:,4,:]
            close_price_input = close_price_input.to(device)
            close_price_target = close_price_target.to(device)

            if(close_price_input.shape[0] != configs["batch_size"]) :
                continue

            output = model(close_price_input)

            loss = loss_fn(output,close_price_target)

            val_losses.append(loss)
            cur_val_loss.append(loss)

        avarage_val_loss = np.mean(np.array(cur_val_loss))

        scheduler.step(avarage_val_loss)
        
        if(best_loss > avarage_val_loss) :

            state = {
                'epoch': epoch,
                'model': model.state_dict(),
                'optimizer': optimizer.state_dict(),
                'scheduler': scheduler.state_dict(),
                'train_loss': avarage_train_loss,
                'val_loss': avarage_val_loss,
                'best_val_loss': best_loss,
            }

            torch.save(state, model_folder + "LSTM1.pth.tar")

    print("train loss : ",avarage_train_loss)
    print("val loss : ",avarage_val_loss)


  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/1410 [00:00<?, ?it/s]

d:\Programming\Pattern_Reg_Final\DatasetLogReturn.py:46: RuntimeWarning: divide by zero encountered in log
  c = np.log(b)
d:\Programming\Pattern_Reg_Final\DatasetLogReturn.py:46: RuntimeWarning: divide by zero encountered in log
  c = np.log(b)
d:\Programming\Pattern_Reg_Final\DatasetLogReturn.py:46: RuntimeWarning: divide by zero encountered in log
  c = np.log(b)
d:\Programming\Pattern_Reg_Final\DatasetLogReturn.py:46: RuntimeWarning: divide by zero encountered in log
  c = np.log(b)
d:\Programming\Pattern_Reg_Final\DatasetLogReturn.py:46: RuntimeWarning: divide by zero encountered in log
  c = np.log(b)
d:\Programming\Pattern_Reg_Final\DatasetLogReturn.py:46: RuntimeWarning: divide by zero encountered in log
  c = np.log(b)
d:\Programming\Pattern_Reg_Final\DatasetLogReturn.py:46: RuntimeWarning: divide by zero encountered in log
  c = np.log(b)
d:\Programming\Pattern_Reg_Final\DatasetLogReturn.py:46: RuntimeWarning: divide by zero encountered in log
  c = np.log(b)
d:\Programming\P

RuntimeError: Expected hidden[0] size (4, 54, 256), got [4, 128, 256]